In [1]:
import cv2
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
from random import shuffle

import os
import shutil

In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import Callback

Using TensorFlow backend.


In [3]:
# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1/255., shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 
val_datagen = ImageDataGenerator(rescale=1/255.)

In [10]:
batch_size = 32
train_generator = train_datagen.flow_from_directory(
        r'data\mnistasjpg\trainingSet\trainingSet',
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        r'data\mnistasjpg\valSet',
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

Found 37000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.


In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [20]:
ratio = 0.2
n = 5000
epochs = 5
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit_generator(train_generator,
          steps_per_epoch= int(n * (1-ratio)) // batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps= int(n * ratio) // batch_size,
          verbose=0)